In [1]:
import socket
import time
from datetime import datetime
import threading
import pyvisa
import numpy as np

In [2]:
'''
Setup communication
'''
# pip install pyvisa, pyvisa-py'
#find ID's with "lsusb"
#then run this as super user
#echo 'SUBSYSTEM=="usb", ACTION=="add", ATTRS{idVendor}=="05e6", ATTRS{idProduct}=="6500" GROUP="tty", MODE="0660"' >> /etc/udev/rules.d/99-com.rules
# sudo usermod -a -G tty pi

rm = pyvisa.ResourceManager()
rm.list_resources('?*')

inst = rm.open_resource('USB0::1510::25856::04440503::0::INSTR')

inst.write('*IDN?')
print(inst.read())

KEITHLEY INSTRUMENTS,MODEL DMM6500,04440503,1.0.04b



In [3]:
'''
set up backend
'''
import importlib
import serverBackend as sb
importlib.reload(sb)

#add the commands we want to use - see manual
multimter_keys = [
    'SENS:FUNC', # "CURR:DC" "VOLT:DC" etc
    'SENS:CURR:DC:RANG', # current range in A, 0.001, 0.1, 1 etc
    'DISP:CURR:DC:DIG', # digits shown on screen
    'SENS:CURR:DC:NPLC', #This command sets the integration time in number of power line cycles (PLCs) on the specified channels.Integration time affects the measurement resolution (for better resolution, use a longer integration time) and measurement speed (for faster measurements, use a shorter integration time).If you omit the optional <ch_list> parameter, this command applies to the currently defined scan list.
    'READ?',
    ]
sb.setup(inst, multimter_keys, 'multimeter.log')
sb.query('*IDN?')

*IDN? KEITHLEY INSTRUMENTS,MODEL DMM6500,04440503,1.0.04b



'KEITHLEY INSTRUMENTS,MODEL DMM6500,04440503,1.0.04b'

In [4]:
print(sb.keys)

['SENS:FUNC', 'SENS:CURR:DC:RANG', 'DISP:CURR:DC:DIG', 'SENS:CURR:DC:NPLC', 'READ?']


In [5]:
sb.set_single_param('SENS:FUNC','"CURR:DC"')
sb.set_single_param('SENS:CURR:DC:RANG','0.001')
sb.set_single_param('SENS:CURR:DC:NPLC','1')

   set  SENS:FUNC: "CURR:DC"
   set  SENS:CURR:DC:RANG: 0.001
   set  SENS:CURR:DC:NPLC: 1


In [6]:
sb.query('READ?') # A

READ? -1.594820E-08



'-1.594820E-08'

In [7]:
sb.set_single_param('DISP:CURR:DC:DIG','6')


   set  DISP:CURR:DC:DIG: 6


In [ ]:
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
port = 5008
host = sb.get_ip() #"127.0.0.1"
serversocket.bind((host, port))
serversocket.listen(5)
print ('server started and listening')
while True:
    clientsocket, address = serversocket.accept()
    sb.server(clientsocket, address)

server started and listening
Current Time = 15:10:24
READ? -1.737748E-08

Current Time = 15:10:27
READ? -8.308879E-09

Current Time = 15:10:29
READ? -1.073950E-08

Current Time = 15:11:06
READ? -5.925868E-09

Current Time = 15:11:07
READ? -7.186710E-09

Current Time = 15:11:08
READ? -8.022097E-09

Current Time = 15:11:10
READ? -1.271457E-09



Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/home/pi/rpi_signal_control/serverBackend.py", line 132, in run
    parameters = data_to_dict(data)
  File "/home/pi/rpi_signal_control/serverBackend.py", line 97, in data_to_dict
    parameters[key_val[0]] = key_val[1]
IndexError: list index out of range


Current Time = 15:11:23
Current Time = 15:11:30
READ? -7.812164E-09

Current Time = 15:15:18
READ? -7.523031E-09

Current Time = 15:15:35
READ? -6.254085E-09

Current Time = 15:15:50
READ? -1.794779E-08

Current Time = 15:16:03
READ? -1.262395E-08

Current Time = 15:18:53
READ? -7.163153E-09

Current Time = 15:20:33
READ? -6.843599E-09

Current Time = 15:20:50
READ? 3.313601E-09

Current Time = 15:20:54
READ? -9.164037E-09

Current Time = 15:21:04
READ? -6.243616E-09

Current Time = 15:21:41
READ? -7.043016E-09

Current Time = 15:22:12
READ? -5.576035E-09

Current Time = 15:22:36
READ? -1.503346E-08

Current Time = 15:23:40
READ? -7.465367E-09

Current Time = 15:24:25
READ? -9.087899E-09

Current Time = 15:27:20
READ? -1.647360E-09

Current Time = 15:28:55
READ? -6.967561E-09

